In [7]:
from .. import oncle_lapsim.toolkit as tl
import sim_notebooks
from toolkit.lap.gps_importer import *
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

raw_track = load_track_from_mat('./Data/TrackMaps/2017_Michigan_Endurance_First4Laps.mat')

ImportError: attempted relative import with no known parent package